In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
import datetime

# Step 1: Load and preprocess the 2021 data
file_path = 'Marrakech Data 2021.xlsx'  # Replace with the correct file path
data_df = pd.read_excel(file_path, sheet_name='Export')

# Convert 'date' to datetime and create additional features
data_df['date'] = pd.to_datetime(data_df['date'])
data_df['day'] = data_df['date'].dt.day
data_df['month'] = data_df['date'].dt.month
data_df['weekday'] = data_df['date'].dt.weekday

# Fill missing values for features
features = ['day', 'month', 'weekday', 'prcp', 'wdir', 'wspd', 'pres']
data_df[features] = data_df[features].fillna(data_df[features].median())

# Prepare input features and target variable
X = data_df[features]
y = data_df['tavg']

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

# Step 2: Build and train the neural network
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=1)

# Step 3: Generate predictions for 2022
future_dates = [datetime.date(2022, 1, 1) + datetime.timedelta(days=i) for i in range(365)]
future_df = pd.DataFrame({
    'date': future_dates,
    'day': [d.day for d in future_dates],
    'month': [d.month for d in future_dates],
    'weekday': [d.weekday() for d in future_dates],
    'prcp': np.random.uniform(data_df['prcp'].min(), data_df['prcp'].max(), len(future_dates)),
    'wdir': np.random.uniform(data_df['wdir'].min(), data_df['wdir'].max(), len(future_dates)),
    'wspd': np.random.uniform(data_df['wspd'].min(), data_df['wspd'].max(), len(future_dates)),
    'pres': np.random.uniform(data_df['pres'].min(), data_df['pres'].max(), len(future_dates))
})

# Normalize future data and predict temperatures
future_X = scaler.transform(future_df[features])
future_predictions = model.predict(future_X)
future_df['tavg_predicted'] = future_predictions

# Step 4: Load actual 2022 data and compare
actual_data_path = 'Marrakech_2022_Data.xlsx'  # Replace with the correct file path
actual_data = pd.read_excel('Marrakech_2022_Data.xlsx')

# Ensure 'date' column is datetime and align data
actual_data['date'] = pd.to_datetime(actual_data['date'])
merged_data = pd.merge(future_df[['date', 'tavg_predicted']], actual_data[['date', 'tavg']], on='date', suffixes=('_predicted', '_actual'))

# Calculate accuracy metrics
mae = mean_absolute_error(merged_data['tavg_actual'], merged_data['tavg_predicted'])
mse = mean_squared_error(merged_data['tavg_actual'], merged_data['tavg_predicted'])
r2 = r2_score(merged_data['tavg_actual'], merged_data['tavg_predicted'])

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.2f}")

# Step 5: Plot the comparison
plt.figure(figsize=(14, 7))
plt.plot(merged_data['date'], merged_data['tavg_actual'], label='Actual Temperature', color='blue', marker='o', linestyle='-', alpha=0.7)
plt.plot(merged_data['date'], merged_data['tavg_predicted'], label='Predicted Temperature', color='orange', marker='x', linestyle='--', alpha=0.7)
plt.title('Comparison of Actual vs Predicted Temperatures for 2022', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Temperature (°C)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Save the comparison results
comparison_output_path = 'Comparison_2022.csv'
merged_data.to_csv(comparison_output_path, index=False)
print(f"Comparison data saved to {comparison_output_path}")


Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 486.0816 - mae: 21.0695 - val_loss: 423.9676 - val_mae: 19.4540
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 470.9853 - mae: 20.7481 - val_loss: 405.6366 - val_mae: 18.9711
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 422.3861 - mae: 19.5587 - val_loss: 371.4385 - val_mae: 18.0358
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 405.2500 - mae: 19.0653 - val_loss: 303.6087 - val_mae: 16.0226
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 312.5654 - mae: 16.4490 - val_loss: 189.6057 - val_mae: 11.9126
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 176.9202 - mae: 11.7556 - val_loss: 72.8433 - val_mae: 6.8191
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 68.6881 - mae: 6.7245 - val_loss: 72.9765 - val_mae: 7.3709
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 52.6723 - mae: 5.8224 - val_loss: 65.3079 - val_mae: 7.0414
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━

FileNotFoundError: [Errno 2] No such file or directory: 'Actual_2022_Data.xlsx'